In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama

# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [2]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [3]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [4]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [17]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too. \n\n"
    user_prompt += website.text
    return user_prompt

In [6]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [7]:
def message_payload(website):
    url = Website(website)
    payload = {
            "model": MODEL,
            "messages": messages_for(url),
            "stream": False
    }
    return payload

In [8]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    payload = message_payload(url)
    response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
    return response.json()['message']['content']

In [9]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [18]:
display_summary("https://www.metatrader5.com/es")

**Resumen del sitio web**

El sitio web Plataforma para fórex y las bolsas de valores: MetaTrader 5 es una plataforma comercial destinada a los traders de forex, acciones y otros activos financieros. La plataforma ofrece una variedad de herramientas y características, incluyendo gráficos, análisis técnico y fundamental, alertas y servicios comerciales.

**Noticias**

* MetaQuotes presentará sus últimas novedades en la iFX Expo de México del 9 al 10 de abril.
* El bróker global regulado ha ampliado significativamente la gama de herramientas disponibles para los tráders con MetaTrader 5 Automations.
* MetaTrader 5 ha sido premiada como la mejor plataforma comercial multimercado en la feria de Ciudad de México.
* La aplicación móvil MetaTrader 5 para iOS ha recibido una actualización con características nuevas, incluyendo objetos de texto para gráficos y temporizador de barras.

**Características destacadas**

* Trading móvil con aplicaciones para iPhone/iPad y Android
* Señales comerciales y copiado de operaciones
* Servicios comerciales y alertas
* Gráficos y análisis técnico y fundamental
* Entorno de desarrollo MQL5 y lenguaje de programación MQL5
* Virtual hosting (VPS) y trading automatizado

**Servicios**

* Descarga de la plataforma MetaTrader 5
* Comprar la plataforma MetaTrader 5
* Conexión a la plataforma web y tradere automatizado
* Acceso a la comunidad de traders y recursos educativos